<a href="https://colab.research.google.com/github/Juliana001/Identificacao-de-queimadas-em-tempo-real-por-imagens-de-satelites/blob/main/C%C3%B3digos%20utilizados%20na%20pesquisa/ICtransformandoLatLon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Apresentacao](https://drive.google.com/uc?export=view&id=1xv_gPp1c0v_pxIIqePqP6YSfs0DuIUQA)


###ABI fixed grid é a informação de projeção presente em praticamente todos os produtos GOES. Ela substitui latitude e longitude. No decorrer da pesquisa, surgiu a necessidade de fazer a conversão do ABI fixed grid para latitude e longitude, uma vez que tornaria o processo de localização mais simples. O código abaixo foi disponibilizado pelo NOAA no endereço: https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_abi_lat_lon.php


Referências:

* https://www.star.nesdis.noaa.gov/atmospheric-composition-training/satellite_data_goes_imager_projection.php#:~:text=Calculating%20Latitude%20and%20Longitude%20from,in%20the%20GOES%2DR%20PUG.

* https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_abi_lat_lon.php

In [1]:
!pip install goes2go

In [34]:
from goes2go import GOES
from goes2go import GOES
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from goes2go.data import goes_nearesttime
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [2]:
g = goes_nearesttime(
    datetime(2025, 9, 25, 16), satellite="goes19", product="ABI-L2-MCMIPF", domain='F',return_as="xarray"
)

 👮🏻‍♂️ File already exists. Do not overwrite: /root/data/noaa-goes19/ABI-L2-MCMIPF/2025/268/16/OR_ABI-L2-MCMIPF-M6_G19_s20252681600192_e20252681609506_c20252681609583.nc
📦 Finished downloading [1] files to [/root/data/noaa-goes19/ABI-L2-MCMIPF].
📚 Finished reading [1] files into xarray.Dataset.


In [29]:
def calculate_degrees(file_id):

    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]  # E/W scanning angle in radians
    y_coordinate_1d = file_id.variables['y'][:]  # N/S elevation angle in radians
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = file_id.variables['goes_imager_projection'].attrs['longitude_of_projection_origin']
    H = file_id.variables['goes_imager_projection'].attrs['perspective_point_height'] + file_id.variables['goes_imager_projection'].attrs['semi_major_axis']
    r_eq = file_id.variables['goes_imager_projection'].attrs['semi_major_axis']
    r_pol = file_id.variables['goes_imager_projection'].attrs['semi_minor_axis']

    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)

    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    #Aqui foi necessário fazer uma pequena alteração do código original, uma vez que os valores negativos retornavam nan
    discriminant = b_var**2 - 4*a_var*c_var
    discriminant = np.maximum(discriminant, 0)  # garante não-negativo
    r_s = (-b_var - np.sqrt(discriminant)) / (2*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)

    # Ignore numpy errors for sqrt of negative number; occurs for GOES-16 ABI CONUS sector data
    np.seterr(all='ignore')

    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)

    return abi_lat, abi_lon

In [30]:
# Print arrays of calculated latitude and longitude

# Call function to calculate latitude and longitude from GOES ABI fixed grid projection data
abi_lat, abi_lon = calculate_degrees(g)

# Print latitude array
print(abi_lat)

# Print max and min of latitude data to check data range
print('The maximum latitude value is', np.max(abi_lat), 'degrees')
print('The minimum latitude value is', np.min(abi_lat), 'degrees')

# Print longitude array
print(abi_lon)

# Print max and min of longitude data to check data range
print('The maximum longitude value is', np.max(abi_lon), 'degrees')
print('The minimum longitude value is', np.min(abi_lon), 'degrees')

[[ 43.57067606  43.58140095  43.59212956 ...  43.59212956  43.58140095
   43.57067606]
 [ 43.56111773  43.57184277  43.58257154 ...  43.58257154  43.57184277
   43.56111773]
 [ 43.55155514  43.56228033  43.57300926 ...  43.57300926  43.56228033
   43.55155514]
 ...
 [-43.55155514 -43.56228033 -43.57300926 ... -43.57300926 -43.56228033
  -43.55155514]
 [-43.56111773 -43.57184277 -43.58257154 ... -43.58257154 -43.57184277
  -43.56111773]
 [-43.57067606 -43.58140095 -43.59212956 ... -43.59212956 -43.58140095
  -43.57067606]]
The maximum latitude value is 81.32564470322127 degrees
The minimum latitude value is -81.32564414818319 degrees
[[-147.92914191 -147.92918442 -147.92922492 ...   -2.07077508
    -2.07081558   -2.07085809]
 [-147.9351089  -147.9351536  -147.93519629 ...   -2.06480371
    -2.0648464    -2.0648911 ]
 [-147.94107428 -147.94112116 -147.94116604 ...   -2.05883396
    -2.05887884   -2.05892572]
 ...
 [-147.94107428 -147.94112116 -147.94116604 ...   -2.05883396
    -2.058878

#O código a seguir serve para incluir os dados de latitude e longitude em g, que é onde estão todos os dados puxados do GOES19 anteriormente

In [31]:
import xarray as xr

# Adiciona latitude e longitude como coordenadas
g = g.assign_coords(latitude=(('y', 'x'), abi_lat))
g = g.assign_coords(longitude=(('y', 'x'), abi_lon))

# Alternativamente, como variáveis
g['latitude'] = (('y', 'x'), abi_lat)
g['longitude'] = (('y', 'x'), abi_lon)

In [32]:
print(g)
print(g['latitude'])
print(g['longitude'])

<xarray.Dataset> Size: 4GB
Dimensions:                                 (y: 5424, x: 5424,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates: (12/43)
  * y                                       (y) float32 22kB 0.1518 ... -0.1518
  * x                                       (x) float32 22kB -0.1518 ... 0.1518
    t                                       datetime64[ns] 8B 2025-09-25T16:0...
    y_image                                 float32 4B 0.0
    x_image                                 float32 4B 0.0
    band_wavelength_C01                     (band) float32 4B 0.47
    ...                                      ...
    dataset_name                            <U74 296B 'OR_ABI-L2-MCMIPF-M6_G1...
    date_created                            <U22 88B '2025-09-25T16:09:58.3Z'
    time_coverage_start                     <U22 88B '2025-09-25T16:00:19.2Z'
    time_coverage_end    